In [1]:
from jaqalpaq.qsyntax import circuit
from jaqalpaq.run import run_jaqal_string, run_jaqal_circuit
from jaqalpaq.qsyntax.qsyntax import Q as Q_obj
from numpy import pi
import numpy as np
from tavis_cummings import Cavity, plot_populations
from plotly import graph_objects as go

In [2]:
def cs(N: int, g: float, kappa: float, initial_populations, t: float) -> float:
    D = np.emath.sqrt(-16 * N * g**2 + kappa**2)
    m = 1 - np.exp(-kappa * t / 4) * ((kappa / D) * np.sinh(D * t / 4) + np.cosh(D * t / 4))
    return initial_populations - m * initial_populations.sum() / N

def thetas(N: int, g: float, kappa: float, initial_populations, t: float) -> list[float]:
    c = cs(N, g, kappa, initial_populations, t)
    theta_1 = np.arccos(c[0])

    theta = []
    for i in range(1, N):
        denom = np.sin(theta_1) * np.prod(np.cos(theta))
        theta.append(np.arcsin(c[i] / denom))
    
    return np.real([theta_1] + theta)

In [3]:
def CNOT(Q: Q_obj, control, target):
    """Implementation of CNOT using native MS gate."""
    Q.Sy(control)
    Q.Sxx(control, target)
    with Q.parallel():
        Q.Sxd(control)
        Q.Sxd(target)
    Q.Syd(control)


def CRy(Q: Q_obj, control, target, angle: float):
    """Controlled Y rotation."""
    Q.Ry(target, -angle / 2)
    CNOT(Q, control, target)
    Q.Ry(target, -angle / 2)
    CNOT(Q, control, target)
    Q.Ry(target, angle)


@circuit
def qmarina(Q: Q_obj, thetas: list[float]):
    """Main QMARINA algorithm. See arxiv:2208.12029"""
    Q.usepulses("qscout.v1.std")
    num_qubits = len(thetas) + 1  # one rotation angle for each emitter qubit + 1 for the cavity/env
    q = Q.register(num_qubits, name="q")

    with Q.subcircuit():
        Q.Px(q[1])  # excite first emitter qubit

        # first emitter is handled oppositely from the others
        CRy(Q, q[1], q[0], 2* thetas[0])  # rotate environment with 1st emitter as control
        CNOT(Q, q[0], q[1])  # CNOT on 1st emitter with environment as control

        for i, theta in enumerate(thetas[1:], start=2):
            CRy(Q, q[0], q[i], 2 * theta)  # rotate ith emitter with environment as control
            CNOT(Q, q[1], q[0])  # CNOT on environment with ith emitter as control


In [4]:
initial_populations = np.array([1, 0])  # populations of emitters in initial state
times = np.linspace(0.01, 5, 100)  # Jaqal doesn't like time=0 for some reason...
populations = {'emitter_0': [], 'emitter_1': [], 'cavity/env': []}  # for collecting the 0

N = 2  # number of emitters
g = 4  # coupling
kappa = 2  # cavity loss

# model for the QuTiP simulation
cavity = Cavity(
    num_emitters=N,
    num_photons=3,
    cavity_freq=5e3,
    emitter_freq=[5e3, 5e3],
    kappa=kappa,
    g=g,
    gamma=0,
)

for t in times:
    theta = thetas(N, g, kappa, initial_populations, t)
    jaqal_circ = qmarina(thetas=theta)
    result = run_jaqal_circuit(jaqal_circ)
    probailities = result.subcircuits[0].probability_by_str
    populations['emitter_0'].append(probailities['110'])
    populations['emitter_1'].append(probailities['101'])
    populations['cavity/env'].append(probailities['100'])


initial_state = cavity.emitter_state(excited_emitter_index=0)
result = cavity.mesolve(initial_state, times)
fig = plot_populations(result)

# The qutip sim doesn't include the environment state with the cavity state
# Let's rescale the cavity state to include the environment
fig.data[0].y = 1 - fig.data[1].y - fig.data[2].y

for key, arr in populations.items():
    fig.add_trace(go.Scatter(x=times, y=arr, name=key, mode="markers"))

fig.show()